Build P3HT bulk simulation 

In [1]:
import os,sys, copy

In [2]:
from streamm import *

In [3]:
%load_ext autoreload
%autoreload 2

Load resources from previous example 

In [10]:
res_tag = 'local'  # Change this to remote to run the calculations remotely 
res_local = resource.Resource(res_tag)
res_local.load_json()

Reading in resource json file res_local.json 


In [11]:
peregrine = resource.Resource('peregrine')
peregrine.load_json()

Reading in resource json file res_peregrine.json 


Load oligomer from previous examples 

In [ ]:
p3ht_n10_esp = buildingblock.Container('p3ht_n10_esp')

In [ ]:
p3ht_n10_esp.read_cply()

In [ ]:
p3ht_n10_esp.lat_cubic(50.0)

In [ ]:
p3ht_n10_esp.guess_oplsa()

In [ ]:
p3ht_n10_esp.bonded_bonds()
p3ht_n10_esp.bonded_angles()
p3ht_n10_esp.bonded_dih()

Place the oligomer into a simulation and add parameters 

In [ ]:
lmp_oligo = lammps.LAMMPS('p3ht_n10_esp_nvt')
lmp_oligo.load_json()

In [ ]:
lmp_oligo.set_resource(peregrine)

In [ ]:
lmp_oligo.properties['scratch'] = lmp_oligo.dir['scratch']

In [ ]:
lmp_oligo.make_dir()

In [ ]:
lmp_oligo.add_strucC(p3ht_n10_esp)

In [ ]:
print lmp_oligo.strucC.lat._matrix

In [ ]:
lmp_oligo.read_param("conj.param")

In [ ]:
lmp_oligo.set_ffparam()

In [ ]:
for p,pt in lmp_oligo.strucC.particles.iteritems(): # properties["charge"] 
    print p,pt.properties["symbol"],pt.properties["fftype"]

Read in templates

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_oligo.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_oligo.dir['launch']

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.load_str('templates','run')

In [ ]:
lmp_oligo.load_str('templates','in')

In [ ]:
lmp_oligo.write_data()

In [ ]:
lmp_oligo.replacewrite_prop('in','input','in','%s.in'%(lmp_oligo.tag))

In [ ]:
lmp_oligo.properties['input_in'] = lmp_oligo.files['input']['in']

In [ ]:
lmp_oligo.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_oligo.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_oligo.tag)
lmp_oligo.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.push()

In [ ]:
lmp_oligo.run()

In [ ]:
lmp_oligo.check()
print lmp_oligo.meta['status']

In [ ]:
lmp_oligo.analysis()
print lmp_oligo.meta['status']

In [ ]:
print lmp_oligo.files['output']

In [ ]:
os.chdir(lmp_oligo.dir['home'])
lmp_oligo.dump_json()
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.meta['status'] = 'finished'

In [ ]:
lmp_oligo.store()

In [ ]:
print lmp_oligo.files['output']

In [ ]:
lmp_oligo.pull()

In [ ]:
os.chdir(lmp_oligo.dir['launch'])

In [ ]:
lmp_oligo.read_data(lmp_oligo.files['output']['data_3'])

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
lmp_oligo.strucC.group_prop('mol','mol')

In [ ]:
lmp_oligo.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_oligo.strucC.tag 

In [ ]:
lmp_oligo.strucC.tag = "%s_nvt"%(lmp_oligo.strucC.tag )

In [ ]:
os.chdir(lmp_oligo.dir['home'])

In [ ]:
lmp_oligo.dump_json()

In [ ]:
lmp_oligo.strucC.write_xyz()

In [ ]:
lmp_oligo.strucC.write_cply()

In [ ]:
p3ht_n10_nvt = copy.deepcopy(lmp_oligo.strucC)

Replicate the oligomer into large simulation cell

In [ ]:
xO = 20

In [ ]:
p3ht_n10_esp_x = buildingblock.Container('p3ht_n10_x%d'%(xO))
p3ht_n10_esp_x.read_cply()

In [ ]:
p3ht_n10_esp_x.lat_cubic(150.0)

In [ ]:
print p3ht_n10_esp_x.lat._matrix

Set seed 

In [ ]:
s = 2984927

In [ ]:
p3ht_n10_esp_x = p3ht_n10_esp_x.add_struc(p3ht_n10_nvt,xO,s)

In [ ]:
p3ht_n10_esp_x.bonded_bonds()
p3ht_n10_esp_x.bonded_angles()
p3ht_n10_esp_x.bonded_dih()

In [ ]:
p3ht_n10_esp_x.tag = 'p3ht_n10_x%d'%(xO)

In [ ]:
p3ht_n10_esp_x.write_xyz()

In [ ]:
p3ht_n10_esp_x.write_cply()

In [ ]:
print p3ht_n10_esp_x.n_bonds,p3ht_n10_esp_x.n_angles,p3ht_n10_esp_x.n_dihedrals

Create LAMMPS calculation to equilibrate bulk system

In [5]:
xO = 20

In [6]:
lmp_bulk = lammps.LAMMPS('p3ht_n10_esp_x%d'%(xO))

In [7]:
lmp_bulk.load_json()

Resource tag found peregrine 
Reading in resource json file res_peregrine.json 


In [ ]:
lmp_bulk.set_resource(peregrine)

In [ ]:
lmp_bulk.properties['scratch'] = lmp_bulk.dir['scratch']

In [ ]:
lmp_bulk.make_dir()

In [ ]:
# lmp_bulk.strucC = buildingblock.Container()

In [ ]:
lmp_bulk.add_strucC(p3ht_n10_esp_x)

In [ ]:
lmp_bulk.read_param("conj.param")

In [ ]:
lmp_bulk.set_ffparam()

In [ ]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_nvt.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_bulk.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

In [ ]:
print lmp_bulk.dir['launch']

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.load_str('templates','run')

In [ ]:
lmp_bulk.load_str('templates','in')

In [ ]:
lmp_bulk.write_data()

In [ ]:
lmp_bulk.replacewrite_prop('in','input','in','%s.in'%(lmp_bulk.tag))

In [ ]:
lmp_bulk.properties['input_in'] = lmp_bulk.files['input']['in']

In [ ]:
lmp_bulk.replacewrite_prop('run','scripts','run','%s.sh'%(lmp_bulk.tag))

In [ ]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_bulk.tag)
lmp_bulk.add_file(file_type,file_key,file_name)

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.push()

In [ ]:
lmp_bulk.run()

In [ ]:
lmp_bulk.check()
print lmp_bulk.meta['status']

In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
print lmp_bulk.dir

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data(lmp_bulk.files['output']['data_3'])

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [ ]:
print lmp_bulk.strucC.tag 

In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

In [ ]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_bulk.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

In [40]:
print lmp_bulk.tag

p3ht_n10_esp_x20


Run Equilibration 

In [8]:
lmp_equ = lammps.LAMMPS('p3ht_n10_x20_equ')

In [12]:
lmp_equ.set_resource(peregrine)

In [13]:
lmp_equ.properties['scratch'] = lmp_equ.dir['scratch']

In [ ]:
lmp_equ.make_dir()

In [14]:
lmp_equ.add_refcalc(lmp_bulk)

In [15]:
for ref_key,ref_calc in lmp_equ.references.iteritems(): 
      print ref_key, ref_calc.tag
            

p3ht_n10_esp_x20 p3ht_n10_esp_x20


In [16]:
lmp_equ.files['input']['data_file'] = lmp_bulk.files['output']['data_3']

In [33]:
lmp_equ.properties['data_file']  = lmp_equ.files['input']['data_file'] 

In [34]:
file_type = 'templates'
file_key = 'in'
file_name = "lammps_equ1.in"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates in lammps_equ1.in templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_equ1.in to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_equ1.in 


In [18]:
file_type = 'templates'
file_key = 'run'
file_name = "lammps_peregrine.pbs"
from_dirkey = 'templates'
to_dirkey = 'launch'
lmp_equ.cp_file(file_type,file_key,file_name,from_dirkey,to_dirkey)

> in cp_file  templates run lammps_peregrine.pbs templates launch
copying /Users/tkemper/Development/streamm_ext/streamm-tools/templates/lammps_peregrine.pbs to /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/lammps_peregrine.pbs 


In [19]:
print lmp_equ.dir['launch']

/Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ/


In [20]:
os.chdir(lmp_equ.dir['launch'])

In [21]:
lmp_equ.load_str('templates','run')

In [35]:
lmp_equ.load_str('templates','in')

In [36]:
lmp_equ.replacewrite_prop('in','input','in','%s.in'%(lmp_equ.tag))

In [24]:
lmp_equ.properties['input_in'] = lmp_equ.files['input']['in']

In [25]:
lmp_equ.replacewrite_prop('run','scripts','run','%s.pbs'%(lmp_equ.tag))

In [26]:
file_type = 'output'
file_key = 'log'
file_name = "%s.log"%(lmp_equ.tag)
lmp_equ.add_file(file_type,file_key,file_name)

In [27]:
os.chdir(lmp_equ.dir['home'])
lmp_equ.dump_json()
os.chdir(lmp_equ.dir['launch'])

{'files': {'templates': {'run': 'lammps_peregrine.pbs', 'in': 'lammps_equ1.in'}, 'input': {'data_file': u'min2.data', 'in': 'p3ht_n10_x20_equ.in'}, 'output': {'log': 'p3ht_n10_x20_equ.log'}, 'data': {}, 'scripts': {'run': 'p3ht_n10_x20_equ.pbs'}}, 'meta': {'date': '2016-11-08T09:33:06.245923', 'status': 'written', 'resource': 'peregrine', 'software': 'lammps'}, 'references': {'p3ht_n10_esp_x20': 'p3ht_n10_esp_x20'}, 'units': {'distance': 'angstroms', 'energy': 'kcal/mol', 'angle': 'radians', 'time': 'ns'}, 'data': {}, 'properties': {'uncompress': 'tar -xzf ', u'scratch': u'/scratch/tkemper/p3ht_n10_x20_equ/', 'compress': 'tar -czf ', u'ppn': 24, 'finish_str': 'Loop time of', u'allocation': u'orgopv', 'run_list': '', u'nproc': 24, 'input_in': 'p3ht_n10_x20_equ.in', u'pmem': 1500, 'compress_sufix': 'tgz', u'feature': u'24core', u'queue': u'short', u'e-mail': u'travis.kemper@nrel.gov', 'comp_key': 'compressed', u'nodes': 1, u'walltime': 4, u'exe_command': u'qsub '}, 'dir': {u'templates': 

In [28]:
ref_calc = lmp_equ.references['p3ht_n10_esp_x20']

In [29]:
print ref_calc.files

{u'templates': {u'run': u'lammps_peregrine.pbs', u'in': u'lammps_nvt.in'}, u'input': {u'compressed': u'p3ht_n10_esp_x20_input.tgz', u'in': u'p3ht_n10_esp_x20.in'}, u'output': {u'log': u'p3ht_n10_esp_x20.log', u'data_3': u'min2.data', u'data_2': u'nvt1.data', u'data_1': u'min1.data', u'compressed': u'p3ht_n10_esp_x20_output.tgz', u'dcd_1': u'nvt1_dump.dcd'}, u'data': {}, u'scripts': {u'run': u'p3ht_n10_esp_x20.sh', u'compressed': u'p3ht_n10_esp_x20_scripts.tgz'}}


In [38]:
lmp_equ.push()

 Resource type ssh 
runnning push function in /Users/tkemper/Development/streamm_ext/streamm-tools/examples/scratch/p3ht_n10_x20_equ 
Compressing and copying input files to scratch directory 
 file_i  min2.data
 file_i  p3ht_n10_x20_equ.in
> compressed_file  p3ht_n10_x20_equ_input.tgz
Compressing and copying templates files to scratch directory 
 file_i  lammps_peregrine.pbs
 file_i  lammps_equ1.in
> compressed_file  p3ht_n10_x20_equ_templates.tgz
Compressing and copying scripts files to scratch directory 
 file_i  p3ht_n10_x20_equ.pbs
> compressed_file  p3ht_n10_x20_equ_scripts.tgz
Copying output of reference calculations p3ht_n10_esp_x20


In [39]:
lmp_equ.run()

Calculation with status written 
Resource type ssh 
Executing run command ssh tkemper@peregrine.hpc.nrel.gov ' cd /scratch/tkemper/p3ht_n10_x20_equ/ ; qsub p3ht_n10_x20_equ.pbs '  


In [42]:
lmp_equ.check()
print lmp_equ.meta['status']

written


In [ ]:
lmp_bulk.analysis()
print lmp_bulk.meta['status']

In [ ]:
print lmp_bulk.files['output']

In [ ]:
os.chdir(lmp_bulk.dir['home'])
lmp_bulk.dump_json()
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.meta['status'] = 'finished'

In [ ]:
lmp_bulk.store()

In [ ]:
lmp_bulk.pull()

In [ ]:
os.chdir(lmp_bulk.dir['launch'])

In [ ]:
lmp_bulk.read_data(lmp_bulk.files['output']['data_3'])

In [ ]:
lmp_bulk.strucC.write_cply()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.group_prop('mol','mol')

In [ ]:
lmp_bulk.strucC.groupsets['mol'].group_pbcs()

In [43]:
print lmp_bulk.strucC.tag 

blank


In [ ]:
lmp_bulk.strucC.tag = "%s_nvt"%(lmp_bulk.strucC.tag )

In [50]:
os.chdir(lmp_bulk.dir['home'])

In [ ]:
lmp_bulk.dump_json()

In [ ]:
lmp_bulk.strucC.write_xyz()

In [ ]:
lmp_bulk.strucC.write_cply()

Analyize bulk properties with streamm

In [44]:
print lmp_equ.tag

p3ht_n10_x20_equ


In [52]:
equ_rdf = calculation.CalculationRes('%s_rdf1'%(lmp_equ.tag))

In [53]:
equ_rdf.set_resource(peregrine)

In [54]:
equ_rdf.add_refcalc(lmp_bulk)

In [55]:
os.getcwd()

'/Users/tkemper/Development/streamm_ext/streamm-tools/examples'